In [40]:
class Pool:
    def __init__(self, x_reserve, y_reserve):
        self.reserves = [0] * 2
        self.reserves[0] = x_reserve
        self.reserves[1] = y_reserve
    
    def K(self):
        return self.reserves[0] * self.reserves[1]
    
    def out_amount(self, in_amount, in_reserve, out_reserve):
        in_with_fee = in_amount * 0.997
        return in_with_fee * out_reserve / (in_with_fee + in_reserve)
    
    def swap(self, in_index, in_amount):
        out_index = (in_index + 1) % 2
        
        in_reserve = self.reserves[in_index]
        out_reserve = self.reserves[out_index]
        out_amount = self.out_amount(in_amount, in_reserve, out_reserve)
        
        self.reserves[in_index] += in_amount
        self.reserves[out_index] -= out_amount
        
        return out_amount

In [41]:
pool = Pool(10, 10)

In [42]:
pool.swap(0, 1)

0.9066108938801491

In [43]:
10 - 100/ (10 + 0.997)


0.9066108938801491

In [44]:
pool = Pool(10, 10)

in_amount = 1
out_amount = 0

out_delta = pool.swap(0, in_amount)
print(f"K: {pool.K()}, out: {out_delta}")
out_amount += out_delta

K: 100.02728016731837, out: 0.9066108938801491


In [45]:
pool = Pool(10, 10)

in_amount = 1
out_amount = 0

out_delta = pool.swap(0, in_amount / 2)
print(f"K: {pool.K()}, out: {out_delta}")
out_amount += out_delta

out_delta = pool.swap(0, in_amount / 2)
print(f"K: {pool.K()}, out: {out_delta}")
out_amount += out_delta

print("total out:", out_amount)

K: 100.01428775539362, out: 0.47482973758155933
K: 100.02792792738373, out: 0.43172226901992017
total out: 0.9065520066014795


In [46]:
pool = Pool(10, 10)

in_amount = 1
out_amount = 0

out_delta = pool.swap(0, in_amount / 3)
print(f"K: {pool.K()}, out: {out_delta}")
out_amount += out_delta

out_delta = pool.swap(0, in_amount / 3)
print(f"K: {pool.K()}, out: {out_delta}")
out_amount += out_delta

out_delta = pool.swap(0, in_amount / 3)
print(f"K: {pool.K()}, out: {out_delta}")
out_amount += out_delta

print("total out:", out_amount)

K: 100.00967835596995, out: 0.32164403006742587
K: 100.01905514238955, out: 0.30156955033355554
K: 100.02814861044506, out: 0.2833183641040162
total out: 0.9065319445049975


In [47]:

in_amount = 20

for n in range(1, 10):
    pool = Pool(10, 10)
    out_amount = 0
    print("n:", n)
    print("")
    for _ in range(n):
        out_delta = pool.swap(0, in_amount / n)
        print(f"K: {pool.K()}, out: {out_delta}")
        out_amount += out_delta

    print("total out:", out_amount)
    print("")

n: 1

K: 100.20040080160321, out: 6.65998663994656
total out: 6.65998663994656

n: 2

K: 100.150225338007, out: 4.99248873309965
K: 100.25047581382081, out: 1.6658287397729894
total out: 6.658317472872639

n: 3

K: 100.12014417300763, out: 3.992791349619543
K: 100.2060350602021, out: 1.712664290657511
K: 100.27288364930163, out: 0.9521149047462257
total out: 6.65757054502328

n: 4

K: 100.1001001001001, out: 3.3266599933266603
K: 100.1752315237429, out: 1.6645784304861946
K: 100.23537274739134, out: 0.9993466662914916
K: 100.2855155051439, out: 0.6665643930575232
total out: 6.65714948316187

n: 5

K: 100.08578781812982, out: 2.85101515584787
K: 100.15255618892243, out: 1.5849539447675502
K: 100.20721466965135, out: 1.009157505309519
K: 100.2534855091171, out: 0.6989701052628654
K: 100.29360295029721, out: 0.5127831904689555
total out: 6.65687990165676

n: 6

K: 100.07505629221917, out: 2.4943707780835624
K: 100.13513737464397, out: 1.4975209794378004
K: 100.1852299896388, out: 0.998846

## Fee-first Uniswap-style pool

The Uniswap V2 pool takes the fee out of the in-token amount, does the swap, and then increase the invariant by adding the fee.  A natural question to ask is what happens if you increase the invariant first by adding the fee and then doing the swap.

The below experiment suggests this would be inefficient as it incentivizes large swaps to be split up in order to reduce the increase of the invariant and thereby get more tokens out.

In [38]:
class FeeFirstPool:
    def __init__(self, x_reserve, y_reserve):
        self.reserves = [0] * 2
        self.reserves[0] = x_reserve
        self.reserves[1] = y_reserve
    
    def K(self):
        return self.reserves[0] * self.reserves[1]
    
    def out_amount(self, in_amount, in_reserve, out_reserve):
        in_reserve_with_fee = in_reserve + 0.003 * in_amount
        in_with_fee = in_amount * 0.997
        return in_with_fee * out_reserve / (in_with_fee + in_reserve_with_fee)
    
    def swap(self, in_index, in_amount):
        out_index = (in_index + 1) % 2
        
        in_reserve = self.reserves[in_index]
        out_reserve = self.reserves[out_index]
        out_amount = self.out_amount(in_amount, in_reserve, out_reserve)
        
        self.reserves[in_index] += in_amount
        self.reserves[out_index] -= out_amount
        
        return out_amount

In [39]:

in_amount = 20

for n in range(1, 10):
    pool = FeeFirstPool(10, 10)
    out_amount = 0
    print("n:", n)
    print("")
    for _ in range(n):
        out_delta = pool.swap(0, in_amount / n)
        print(f"K: {pool.K()}, out: {out_delta}")
        out_amount += out_delta

    print("total out:", out_amount)
    print("")

n: 1

K: 100.60000000000001, out: 6.6466666666666665
total out: 6.6466666666666665

n: 2

K: 100.3, out: 4.985
K: 100.45044999999999, out: 1.6666516666666666
total out: 6.651651666666667

n: 3

K: 100.19999999999999, out: 3.988000000000001
K: 100.32024, out: 1.7125611428571421
K: 100.40622877714286, out: 0.9525645645714285
total out: 6.653125707428572

n: 4

K: 100.15, out: 3.3233333333333333
K: 100.25014999999999, out: 1.6641591666666666
K: 100.3253376125, out: 0.9994939955000001
K: 100.38553281506749, out: 0.6668290773310834
total out: 6.653815572831084

n: 5

K: 100.12, out: 2.8485714285714288
K: 100.20581714285714, out: 1.5844387301587302
K: 100.27262102095237, out: 1.009143431226551
K: 100.32731517787289, out: 0.69910351858664
K: 100.37362009257036, out: 0.5129555550376373
total out: 6.654212663580988

n: 6

K: 100.10000000000001, out: 2.4925
K: 100.17507500000002, out: 1.4969955
K: 100.23518004500002, out: 0.9987454977499999
K: 100.28529763502252, out: 0.7138176750347499
K: 100.3